In [1]:
#Only run in colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/flywire

ModuleNotFoundError: No module named 'google.colab'

In [19]:
import numpy as np
import scipy as sp
from utility import get_best_connected, invert, score, random_graph_dist, shuffle_graph, shuffle_cycles, get_cycles, random_extension, shuffle_map
from algorithms import move_swap_optimization
from data_loader import M,F, benchmark_mapping
rng = np.random.default_rng(0)

In [4]:
A=M.todense()
B=F.todense()
N=A.shape[0]
A[np.arange(N),np.arange(N)]=0
B[np.arange(N),np.arange(N)]=0

In [10]:
from algorithms import move_swap_optimization

m=800
n=A.shape[0]
#mapping_M = rng.choice(n,m,replace=False)
#mapping_F = rng.choice(n,m,replace=False)

def penalty_gradient(x):

    return (x>0)*10000000

top_score = 0

for i in range(2):

    outer_unchanged = True

    if i % 2 == 0:
        G = B[mapping_F[:,None],mapping_F[None,:]]
        H = A
        mapping = mapping_M.copy()
    else:
        G = A[mapping_M[:,None],mapping_M[None,:]]
        H = B
        mapping = mapping_F.copy()

    mapping, new_top_score = move_swap_optimization(G,H,mapping,penalty_gradient=penalty_gradient,rng=rng,verbose=True)
    
    if i % 2 == 0:
        mapping_M = mapping.copy()
    else:
        mapping_F = mapping.copy()

    print(" "*100,end="\r")
    print(f"{i=}, {new_top_score=}"+" "*20,end="\n")

    if new_top_score == top_score:
        break
    else:
        top_score = new_top_score

mapping = random_extension(n,mapping_F,mapping_M,rng)

i=0, new_top_score=308689                                                                           
i=1, new_top_score=309361                                                                           


In [477]:
n=500
G = random_graph_dist(n,p=0.1,sample_source=lambda : int(1/np.sqrt(rng.random())),rng=rng)
H,shuffle = shuffle_graph(G,rng=rng)

sigma = rng.permutation(n)
tau = rng.permutation(n)

penalty_gradient = lambda x: (x>0)*10000000
score(G,G,np.arange(n))

41800

In [ ]:
for i in range(10):
    sigma, sigma_score = move_swap_optimization(G,H,sigma,penalty_gradient=penalty_gradient,max_epochs=10,verbose=False,randomize_argmax=True)
    tau, tau_score = move_swap_optimization(G,H,tau,penalty_gradient=penalty_gradient,max_epochs=10,verbose=False,randomize_argmax=True)
    rho = invert(sigma)[tau]
    print(sigma_score,tau_score, len(get_cycles(rho)))

    if sigma_score > tau_score:
        tau = rng.permutation(n)#sigma[shuffle_cycles(rho)]
    else:
        sigma = rng.permutation(n)#tau[shuffle_cycles(rho)]

In [489]:
def find_preferences(G,H,iterations = 100, method=None, rng = None, verbose=True):

    n = G.shape[0]
    result_matrix = np.zeros((n,n),dtype=int)

    if rng is None:
        rng = np.random.default_rng()

    for k in range(iterations):

        mapping = rng.permutation(n)
        mapping, _ = method(G,H,mapping)

        np.add.at(result_matrix,(np.arange(n),mapping),1)
        if verbose:
            print(f"{k=}"+" " * 20,end="\r")

    return result_matrix

method = lambda G,H,mapping : move_swap_optimization(G,H,mapping,penalty_gradient=penalty_gradient,max_epochs=3,verbose=False,randomize_argmax=True)
result_matrix = find_preferences(G,H,iterations=20,method=method)

best=result_matrix.argmax(axis=1)
best_scores = result_matrix[np.arange(n),best]
mapping,final_score=move_swap_optimization(G,H,best,penalty_gradient=penalty_gradient,max_epochs=10,verbose=False,randomize_argmax=True)
final_score

41800

In [8]:
mapping0 = np.load("mappings\Mapping-14-12-24 (5.82 Mio).npy")
mapping1 = np.load("mappings\Mapping-09-01-25 sample1.npy")
mapping2 = np.load("mappings\Mapping-09-01-25 sample2.npy")
mapping3 = np.load("mappings\Mapping-09-01-25 sample3.npy")
G=B
H=A

In [20]:
def penalty_gradient(x):

    return (x>0)*10000000

stable = (mapping0==mapping1) & (mapping1==mapping2) & (mapping2==mapping3)
mapping = shuffle_map(mapping0,stable)
move_swap_optimization(G,H,mapping,penalty_gradient=penalty_gradient,max_epochs=5,verbose=True,randomize_argmax=True)

KeyboardInterrupt: 

In [ ]:
rho_inverse = invert(rho)
rho_pow = np.arange(n)
rho_inv_pow = np.arange(n)

distance = n*np.ones(n,dtype=int)

for i in range(n):

    hit = np.logical_or(shuffle == sigma[rho_pow],shuffle==sigma[rho_inv_pow])
    distance[hit] = np.minimum(distance[hit],i)
    rho_pow = rho[rho_pow]
    rho_inv_pow = rho_inverse[rho_inv_pow]

np.bincount(distance),get_cycles(rho)